In [1]:
%matplotlib inline

from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

import sys 
sys.path.append(os.path.abspath("..\examples"))

import utils; reload(utils)
from utils import plots

Using Theano backend.


In [13]:
pwd = str(os.getcwd()).split("\\")
pwd.pop()
pwd.pop()
spwd = "/".join(pwd)
# path = spwd + "/data/dogscats/"
path = spwd + "/data/dogscats/sample/"

In [14]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=8

In [15]:
# Import our class, and instantiate
import vgg16; reload(vgg16)
from vgg16 import Vgg16

## 微调(finetune)模型

每次抓取一些图片用于训练(train)和验证(valid).  
对于vgg, 样本必须分类放好, 比如:  
data\train\cats\*.jpg  
data\train\dogs\*.jpg  

In [16]:
vgg = Vgg16()

batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
Epoch 1/1
16/16 [==============================] - 13s - loss: 2.1968 - acc: 0.3125 - val_loss: 0.8710 - val_acc: 0.8750


## 生成结果

In [6]:
# 注意这里test目录下还要有一层目录, 再存放待识别的图片, 不然会找不到class
batches, preds = vgg.test(path+'test/', batch_size = batch_size*2)

Found 12 images belonging to 1 classes.


In [8]:
print(preds)

[[  5.7575e-02   9.4242e-01]
 [  9.9988e-01   1.1977e-04]
 [  9.7367e-01   2.6330e-02]
 [  3.0852e-02   9.6915e-01]
 [  9.1527e-03   9.9085e-01]
 [  1.9234e-02   9.8077e-01]
 [  1.4967e-01   8.5033e-01]
 [  9.9999e-01   1.0702e-05]
 [  9.9536e-01   4.6429e-03]
 [  9.9666e-01   3.3387e-03]
 [  9.9993e-01   7.3736e-05]
 [  9.8580e-01   1.4205e-02]]
